Post process data generated by `generate_data.ipynb`.
Concatenate distance matrix data and put in Pandas dataframe.

In [22]:
from datetime import datetime, timedelta
from pprint import pprint
import os
import json
import numpy as np
import pandas as pd

Load data object.
A json file.

In [23]:
base_path = os.path.join('C:\\', 'Users', 'glenn', 'src', 'pycommute')
data_directory = os.path.join(base_path, 'data')
data_file = os.path.join(data_directory, 'test_data.json')
with open(data_file, 'r') as f:
    data_object = json.load(f)

Get relevant info from data object

In [24]:
distance_matrices_transit = data_object['distance_matrices_transit']
distance_matrices_driving = data_object['distance_matrices_driving']
origins_batches = data_object['origins_batches']
destinations_geocodes = data_object['destinations_geocodes']


In [25]:
destination_coords = [
    (geocode['geometry']['location']['lat'], geocode['geometry']['location']['lng']) for geocode in destinations_geocodes
]

Create pandas table. Columns are:
origin coords,
origin address,
destination coords,
destination address,
travel mode,
distance text,
distance value,
duration text,
duration value,
status,

Function that takes a batch and creates a list of dictionaries that correspond to a row in the dataframe.

In [30]:
def entries_from_batch(batch_matrix, mode, batch_origins, destination_coords):
    entries = []
    assert batch_matrix['status'] == 'OK', "Batch status isn't `OK`. It's {}.".format(batch_matrix['status'])
    destination_addresses = batch_matrix['destination_addresses']
    origin_addresses = batch_matrix['origin_addresses']
    rows = batch_matrix['rows']
    
    assert len(origin_addresses) == len(batch_origins), "Number of origin addresses ({}) doesn't correspond to number of origin coordinates in `batch_origins` ({}).".format(len(origin_addresses), len(batch_origins))
    assert len(destination_addresses) == 2, "Number of destination addresses ({}) isn't 2.".format(len(destination_addresses))

    for row, origin_address, origin_coordinate in zip(rows, origin_addresses, batch_origins):
        elements = row['elements']
        assert len(elements) == 2, "Number of elements ({}) isn't 2".format(len(elements))
        for element, destination_address, destination_coordinate in zip(elements, destination_addresses, destination_coords):
            entry = {
                'origin coords': tuple(origin_coordinate),
                'origin address': origin_address,
                'destination coords': destination_coordinate,
                'destination address': destination_address,
                'travel mode': mode,
                'distance text': element['distance']['text'],
                'distance value': element['distance']['value'],
                'duration text': element['duration']['text'],
                'duration value': element['duration']['value'],
                'status': element['status'],
            }
            entries.append(entry)
    return entries